# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

## ! Use NBViewer !

To view the Visualizations in this notebook, please use NbViewer:

[Arvato Project Workbook.ipynb](https://nbviewer.jupyter.org/github/lewi0332/arvato_financial_customer_segmentation/blob/master/Arvato%20Project%20Workbook.ipynb)

In [2]:
from sklearnex import patch_sklearn
patch_sklearn()

# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import pickle as pkl
from pandas_profiling import ProfileReport
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb


import plotly.graph_objects as go 
import plotly.express as px
pd.options.plotting.backend = "plotly"

import fohr_theme_light

import plotly.io as pio
pio.renderers.default = "notebook_connected"
import chart_studio.plotly as py


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Udacity Workspace not Useable. 

First step in this project is to move the data out of the Udacity workspace as the IDE on the site was not nearly capable of working on this project. The Udacity supplied IDE took 22 minutes just to load the data into memory. Thus, it clearly did not have the resources to perform machine learning tasks on such a large data set. 

So, the first step in this project was to get the data out of the Udacity Workspace and download it to my local maching. The workspace was so underpowered that I could not even convert the files to Parquet first to reduce network traffic and ease the burden of downloading to my local computer. In fact, it could not even handle the conda install to _attempt_ to get the libraries needed to save the file in Parquet format.

**Convert Udacity Supplied data into Parquet format**. Once the data was on my local machine I decided to conver the data to Parquet anyway. Parquet is binary format that is not only compressed (uses 10x less space) but maintains data type. While Pandas has logic built in to determine the data type, in this case it was confused when given multiple data types in a single column and throws an error. Not only do I perfer to use parquet files to CSV for these and other reasons such as future compatibility with Spark, but, in this case, by converting to Parquet now I solve this datatype issue once. All future attempts to load the data in order to work on this project will be quicker and datatype-error free.

1.  Fix the two columns in each csv file with multiple datatypes - The columns are categorical in nature and use an integer to label each category. As Integers are lightweight and respond to Sklearn's categorical features, I will convert the columns to integers 

2. Store the resulting dataframes as parquet files to use less space on my local machine and load faster on subsequent working sessions. 

In [3]:
''' 
Items in this cell were run once to convert the data files into a Parquet format. 

Two columns had to be pre-cleaned in order to store the parquet file in the correct 
data type. This is a key component of the project. Therefore I left this convert cell
to show the steps needed and will later clean more columns in a similar way.
'''

# # load in the data
# azdias = pd.read_csv('Udacity_AZDIAS_052018.csv')
# customers = pd.read_csv('Udacity_CUSTOMERS_052018.csv')

# # According to data map supplied "-1" means unknown. Fill non-integer numbers with np.nan and convert to float
# azdias.CAMEO_DEUG_2015 = azdias.CAMEO_DEUG_2015.replace('X', np.NaN).astype(float)
# azdias.CAMEO_INTL_2015 = azdias.CAMEO_INTL_2015.replace('XX', np.NaN).astype(float)

# # According to data map supplied "-1" means unknown. Fill non-integer numbers with np.nan and convert to float
# customers.CAMEO_DEUG_2015 = customers.CAMEO_DEUG_2015.replace('X',  np.NaN).astype(float)
# customers.CAMEO_INTL_2015 = customers.CAMEO_INTL_2015.replace('XX',  np.NaN).astype(float)

# azdias.to_parquet('Udacity_AZDIAS_052018.parquet')
# customers.to_parquet('Udacity_CUSTOMERS_052018.parquet')

' \nItems in this cell were run once to convert the data files into a Parquet format. \n\nTwo columns had to be pre-cleaned in order to store the parquet file in the correct \ndata type. This is a key component of the project. Therefore I left this convert cell\nto show the steps needed and will later clean more columns in a similar way.\n'

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [4]:
azdias = pd.read_parquet('Udacity_AZDIAS_052018.parquet')

In [5]:
# profile = ProfileReport(azdias, minimal=True)

In [6]:
# profile.to_file("output.html")

## Pandas Profile report
One of my favorite tools for EDA is the Pandas Profiling library. This library automates the EDA tasks data scientists take on each day. The report outlines the distrobution of each column, its missing values, its data type and some very helpful tools in categorical columns. 

**In the real world** I would do most of my learning about the data and make most decision here with this html document. However, for the scope of this project, it is difficult to show the process with this tool. Thus I will create a few visuals to show highlight *some* of the decisions. 

![Data Profile html visual](pandasprofile.png)
![categorical data](pandasprofilecategorical.png)

### Example

In the first visual for the `LNR` feature, Pandas Profiler has clearly shown that this feature has unique values for all rows and is likely an ID or another type of value that will not help our model. It will be dropped from the dataset before clustering. 


## Part 0.1
Read in the customer data file.  These two files will be used in comparison after creating clusters from the data. I will check for differences in the data. 

In [7]:
customers = pd.read_parquet('Udacity_CUSTOMERS_052018.parquet')

In [8]:
print(azdias.shape)
print(customers.shape)

(891221, 366)
(191652, 369)


The project brief mentioned the three additional columns in the customers dataset. However we need to check to see that the remaining 366 columns are the same. Here I use Python `.intersection()` method to compare two sets of column names.

In [9]:
print(len(set(azdias.columns).intersection(customers.drop(['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'], axis=1).columns)))
print(len(set(customers.drop(['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'], axis=1).columns).intersection(azdias.columns)))


366
366


## Part 0.2 Read in Data Dictionary - Convert unknown values to NaN

Next I will better understand the columns with information from the data dictionary files `DIAS Attributes - Values 2017.xlsx` and `DIAS Information Levels - Attributes 2017.xlsx`. In these files we can learn more about the meanings of each value code. Certain value codes represent `unknown` or `missing`. These values were not collected or completely unknown. Our machine learning algorithms will respond better to `NaN` values in their place. Thus, I must discover and replace all value codes that are `unknown` or `missing`. 

In [10]:
values = pd.read_excel('DIAS Attributes - Values 2017.xlsx', sheet_name='Tabelle1', header=1, engine='openpyxl')

In [11]:
values.head(10)

,Unnamed: 0,Attribute,Description,Value,Meaning
0,NaN,AGER_TYP,best-ager typology,-1,unknown
1,NaN,NaN,NaN,0,no classification possible
2,NaN,NaN,NaN,1,passive elderly
3,NaN,NaN,NaN,2,cultural elderly
4,NaN,NaN,NaN,3,experience-driven elderly
5,NaN,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,NaN,NaN,NaN,1,< 30 years
7,NaN,NaN,NaN,2,30 - 45 years
8,NaN,NaN,NaN,3,46 - 60 years
9,NaN,NaN,NaN,4,> 60 years


Give the size of the dataset, perhaps I can use a bit of string matching to find each attributes specfic code and programitically remove these values. 

I will first build a list of only `attribute` and `value` that can be stored and reused in future data cleaning. 

With these data dictionary documents open in Excel, a quick visual scan helped me see that there are basically 4 distinct string values that represent an unknown value: 'unknown', 'unknown / no main age detectable', 'no transaction known', and 'no transactions known'

In [12]:
# Forward fill the attribute values. 
values['Attribute'] = values['Attribute'].ffill()

In [13]:
value_unknown_codes = values.loc[(values['Meaning'].str.contains(pat='unknown', case=False, regex=False, na=False)) | (values['Meaning'].str.contains(pat='no transaction', case=False, regex=False, na=False))]
value_unknown_codes.head(10)

,Unnamed: 0,Attribute,Description,Value,Meaning
0,NaN,AGER_TYP,best-ager typology,-1,unknown
5,NaN,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
11,NaN,ALTER_HH,main age within the household,0,unknown / no main age detectable
33,NaN,ANREDE_KZ,gender,"-1, 0",unknown
40,NaN,BALLRAUM,distance to next urban centre,-1,unknown
48,NaN,BIP_FLAG,business-flag indicating companies in the buil...,-1,unknown
51,NaN,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,-1,unknown
105,NaN,CAMEO_DEUINTL_2015,CAMEO classification 2015 - international typo...,-1,unknown
131,NaN,CJT_GESAMTTYP,customer journey typology,0,unknown
138,NaN,D19_BANKEN_ANZ_12,transaction activity BANKS in the last 12 months,0,no transactions known


This looked like it worked. Test for all types. 

In [14]:
value_unknown_codes['Meaning'].unique()

array(['unknown', 'unknown / no main age detectable',
       'no transactions known', 'no transaction known'], dtype=object)

That is all four of the unknown meanings that I could see in the file. It looks like our string matching was successful and we are able to parse and retain all of them.

Next, the values on the unknown fields are sometimes multiple integers seperated by a comma. We will need to seperate these and give them their own row as both values will need to be convertied to NaN values.

In [15]:
value_unknown_codes['Value'].values[1]

'-1, 0'

In [16]:
value_unknown_codes = value_unknown_codes[['Attribute']].join(value_unknown_codes['Value'].astype('str').str.split(',', expand=True)).melt(id_vars='Attribute').drop('variable', axis=1)

In [17]:
value_unknown_codes.dropna(inplace=True)

In [18]:
value_unknown_codes.describe()

,Attribute,value
count,380,380
unique,289,5
top,KBA05_MOD2,-1
freq,2,230


In [19]:
value_unknown_codes['value'].unique()

array(['-1', '0', '10', ' 0', ' 9'], dtype=object)

This file will now be used in a future data cleaning process to convert missing values to proper `np.nan` values

In [20]:
value_unknown_codes.value = value_unknown_codes.value.astype('int')

In [21]:
azdias.isna().sum().sum()

33493669

In [22]:
azdias.isna().sum().index

Index(['LNR', 'AGER_TYP', 'AKT_DAT_KL', 'ALTER_HH', 'ALTER_KIND1',
       'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4', 'ALTERSKATEGORIE_FEIN',
       'ANZ_HAUSHALTE_AKTIV',
       ...
       'VHN', 'VK_DHT4A', 'VK_DISTANZ', 'VK_ZG11', 'W_KEIT_KIND_HH',
       'WOHNDAUER_2008', 'WOHNLAGE', 'ZABEOTYP', 'ANREDE_KZ',
       'ALTERSKATEGORIE_GROB'],
      dtype='object', length=366)

In [23]:
def plot_missing_values(dff, renderer='notebook_connected'):
    """
    Visualize the missing values per column in a dataset

    INPUT  - dff : Pandas Dataframe, 
            renderer : Choice of plotly rendering tools. Default of "notebook_connected".capitaliz
                        Other options include "chrome", "firefox", "jpg" etc.  

    OUTPUT - Ploty chart
    """
    temp_missing_values = dff.isna().sum()
    temp_missing_values.sort_values(ascending=False, inplace=True)
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=temp_missing_values.index,
        y=temp_missing_values/len(dff)
    ))
    fig.update_yaxes(title='Rate of Missing Values', tickformat='%')
    fig.update_xaxes(title = 'Column name')
    fig.update_layout(title= "Missing Values per Column")
    return fig

In [24]:
fig = plot_missing_values(azdias)
fig.show()
# py.plot(fig, filename = 'udsnd_1_missing', auto_open=False)

In [25]:
missing_attributes = []
for idx, row in value_unknown_codes.iterrows():
    try:
        azdias.loc[azdias[row['Attribute']] == row['value'], row['Attribute']] = np.nan
    except KeyError:
        missing_attributes.append(row['Attribute'])

In [26]:
azdias.isna().sum().sum()

51863595

Plot the missing values again after converting any of the explicit value codes to a NaN value.

In [27]:
fig = plot_missing_values(azdias)
fig.show()

Lets look at which attributes were in the data dictionary, but not in our `azdias` dataset:

In [28]:
# Items in the attribute dictionary that are not represented in the dataset:
missing_attributes

['BIP_FLAG',
 'CAMEO_DEUINTL_2015',
 'D19_BANKEN_DIREKT_RZ',
 'D19_BANKEN_GROSS_RZ',
 'D19_BANKEN_LOKAL_RZ',
 'D19_BANKEN_REST_RZ',
 'D19_BEKLEIDUNG_GEH_RZ',
 'D19_BEKLEIDUNG_REST_RZ',
 'D19_BILDUNG_RZ',
 'D19_BIO_OEKO_RZ',
 'D19_BUCH_RZ',
 'D19_DIGIT_SERV_RZ',
 'D19_DROGERIEARTIKEL_RZ',
 'D19_ENERGIE_RZ',
 'D19_FREIZEIT_RZ',
 'D19_GARTEN_RZ',
 'D19_HANDWERK_RZ',
 'D19_HAUS_DEKO_RZ',
 'D19_KINDERARTIKEL_RZ',
 'D19_KK_KUNDENTYP',
 'D19_KOSMETIK_RZ',
 'D19_LEBENSMITTEL_RZ',
 'D19_LOTTO_RZ',
 'D19_NAHRUNGSERGAENZUNG_RZ',
 'D19_RATGEBER_RZ',
 'D19_REISEN_RZ',
 'D19_SAMMELARTIKEL_RZ',
 'D19_SCHUHE_RZ',
 'D19_SONSTIGE_RZ',
 'D19_TECHNIK_RZ',
 'D19_TELKO_MOBILE_RZ',
 'D19_TELKO_REST_RZ',
 'D19_TIERARTIKEL_RZ',
 'D19_VERSAND_REST_RZ',
 'D19_VERSICHERUNGEN_RZ',
 'D19_VOLLSORTIMENT_RZ',
 'D19_WEIN_FEINKOST_RZ',
 'GEOSCORE_KLS7',
 'HAUSHALTSSTRUKTUR',
 'KBA13_CCM_1400_2500',
 'SOHO_FLAG',
 'WACHSTUMSGEBIET_NB',
 'GEOSCORE_KLS7',
 'HAUSHALTSSTRUKTUR',
 'WACHSTUMSGEBIET_NB']

Importantly, we must look to see which attributes are in our dataset that are **not** in our data dictionary. 

These items we will not know their meaning. Thus, we do not know if there is a `missing` or `unknown` coded value

In [29]:
azdias_attr = list(azdias.columns)
values_attr = values.Attribute.unique()
azdias_missing_attr = [element for element in azdias_attr if element not in values_attr]

In [30]:
# 94 columns in azdias that we do not find in our data dictionary
len(azdias_missing_attr)

94

In [31]:
# Perhaps these missing attributes are of a specific type? 
azdias[azdias_missing_attr].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Data columns (total 94 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   LNR                         891221 non-null  int64  
 1   AKT_DAT_KL                  817722 non-null  float64
 2   ALTER_KIND1                 81058 non-null   float64
 3   ALTER_KIND2                 29499 non-null   float64
 4   ALTER_KIND3                 6170 non-null    float64
 5   ALTER_KIND4                 1205 non-null    float64
 6   ALTERSKATEGORIE_FEIN        628274 non-null  float64
 7   ANZ_KINDER                  817722 non-null  float64
 8   ANZ_STATISTISCHE_HAUSHALTE  798073 non-null  float64
 9   ARBEIT                      794005 non-null  float64
 10  CAMEO_INTL_2015             791869 non-null  float64
 11  CJT_KATALOGNUTZER           886367 non-null  float64
 12  CJT_TYP_1                   886367 non-null  float64
 13  CJT_TYP_2     

Looks like there were only 2 columns of type "object". All others are `float` or `int` 



In [32]:
azdias[azdias_missing_attr].isna().sum().sum()

9421748

In [33]:
fig = plot_missing_values(azdias[azdias_missing_attr])
fig.show()
# py.plot(fig, filename = 'udsnd_2_missing', auto_open=False)

This presents an interesting challenge about what to do with these columns. Given we do not know the meaning of the coded values, it is possible that some features have values that are unknown but represented with a digit

I could see above that there were 5 unique value codes associated with being unknown: `[-1,  0, 10,  9]`. While I can see these codes in the data in the columns that we have no values for, it is not **certain** that they mean the same thing and we could be losing data if we choose to convert them to `NaN`. 

**The real world** - at this point in a real on-the-job scenario, I would stop the process and communicate with someone in the company to solve for this missing information as it might greatly increase our accuracy. 

In this case, I might have to test both pathways: 
  
  1. Completely Remove the columns I don't understand as the inclusion of `NaN` values may pollute the results.
  2. Convert the value codes that seem to match the `unknown` of other columns.  

In [34]:
azdias[azdias_missing_attr].head()

,LNR,AKT_DAT_KL,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_KINDER,ANZ_STATISTISCHE_HAUSHALTE,ARBEIT,...,STRUKTURTYP,UMFELD_ALT,UMFELD_JUNG,UNGLEICHENN_FLAG,VERDICHTUNGSRAUM,VHA,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11
0,910215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,910220,9.0,NaN,NaN,NaN,NaN,21.0,0.0,12.0,3.0,...,2.0,3.0,3.0,1.0,0.0,0.0,4.0,8.0,11.0,10.0
2,910225,9.0,NaN,NaN,NaN,NaN,17.0,0.0,7.0,3.0,...,3.0,2.0,5.0,0.0,1.0,0.0,2.0,9.0,9.0,6.0
3,910226,1.0,NaN,NaN,NaN,NaN,13.0,0.0,2.0,2.0,...,1.0,4.0,5.0,0.0,0.0,1.0,0.0,7.0,10.0,11.0
4,910241,1.0,NaN,NaN,NaN,NaN,14.0,0.0,3.0,4.0,...,3.0,4.0,3.0,0.0,1.0,0.0,2.0,3.0,5.0,4.0


I will attempt something perhaps a little onthrodox. I will try the second option of keeping each column we don't know, but take a closer look at the columns values to see if we can make a more informed decision about what might be the liekly value code that represents `unknown`. 

First, We can see from the visual above there are many columns that have significant `NaN` values already. I will assume that in these cases the data was collected in a way that left the value empty instead of assigning unknown to a code. For instance, `Alter_Kind` describes Other childern and clearly increases in missing values as the number grows. Fewer families have 4 childern than 3, etc.

The remaining columns (those with less than 50K `nan` values) are suspect. For these columns, I will look at the distrobution of values by name groups. Perhaps we can see a pattern and one that matchs with our previous codes. 

In [35]:
temp_missing_values = azdias[azdias_missing_attr].isna().sum()
temp_missing_values = temp_missing_values[temp_missing_values<50_000]

**D19_**

This name group has the most features in view. From the Attributes .xlsx file I learn this data contains unique data regarding the mail-order activity of consumers, differentiated. I will first examine all of these values to see if we can find an obvious place to start when assuming which value code might represent `unkown`. 

In [36]:
D19_missing = temp_missing_values[temp_missing_values.index.str.contains('D19_')]

In [37]:
D19_missing

D19_BANKEN_DIREKT          0
D19_BANKEN_GROSS           0
D19_BANKEN_LOKAL           0
D19_BANKEN_REST            0
D19_BEKLEIDUNG_GEH         0
D19_BEKLEIDUNG_REST        0
D19_BILDUNG                0
D19_BIO_OEKO               0
D19_BUCH_CD                0
D19_DIGIT_SERV             0
D19_DROGERIEARTIKEL        0
D19_ENERGIE                0
D19_FREIZEIT               0
D19_GARTEN                 0
D19_HANDWERK               0
D19_HAUS_DEKO              0
D19_KINDERARTIKEL          0
D19_KONSUMTYP_MAX          0
D19_KOSMETIK               0
D19_LEBENSMITTEL           0
D19_NAHRUNGSERGAENZUNG     0
D19_RATGEBER               0
D19_REISEN                 0
D19_SAMMELARTIKEL          0
D19_SCHUHE                 0
D19_SONSTIGE               0
D19_TECHNIK                0
D19_TELKO_MOBILE           0
D19_TELKO_REST             0
D19_TIERARTIKEL            0
D19_VERSAND_REST           0
D19_VERSI_DATUM            0
D19_VERSI_OFFLINE_DATUM    0
D19_VERSI_ONLINE_DATUM     0
D19_VERSICHERU

Here are all the columns that begin with `D19_`, do not have explicit information about which value code might represent `unknown` and have less thna 50,000 `NaN` values in across all rows of the dataset. 

After another visual scan of the `DIAS Information Levels - Attributes 2017.xlsx` file. I can see that all of these `D19_` columns match an Information Level called `125 x 125 grid` **HOWEVER**, they are all missing `_RZ` attached to the column name. Further, in the `DIAS Attributes - Values 2017.xlsx` file there are in fact information about the missing value codes for those columns with the `_RZ` suffix. 

While this is another unorthodox assumption, a test could be made using the information about the value code for `unknown` values from the near matching column names. Simply droping the `_RZ` suffix from teh `DIAS Attributes - Values 2017.xlsx` list would match up the column names for nearly all but 4 of our missing `D19` numbers. 

4 `D19_` categories that would still remain. 

- D19_KONSUMTYP_MAX
- D19_VERSI_DATUM
- D19_VERSI_OFFLINE_DATUM
- D19_VERSI_ONLINE_DATUM

**`D19_VERSI_*`** 

Each of these three columns have nearly identical Matches for `D19_VERSAND_*`. Later I will create a change to the unknowns list to match these columns as well. 



In [38]:
D19_missing = azdias[D19_missing.index].astype('float').stack().value_counts()
D19_missing.sort_index().plot.bar()

This view shows us that all value codes are in a similar range to the other known `D19_` columns and the distribution of `0` values also mirrors that of the known group of `D19_` columns that had missing values. Thus, I **will** attempt to remove the `_RZ` suffix from the `Attribute` column in final data clean to match these columns with the nearly identical columns in the `DIAS Attributes - Values 2017.xlsx` file.  

In [39]:
# Remove _RZ suffix from Attributes and re-run our for loop to 
# replace value codes known to represent 'missing' with np.nonlocal

value_unknown_codes.loc[value_unknown_codes['Attribute'].str.endswith('_RZ'), 'Attribute'] = value_unknown_codes.loc[value_unknown_codes['Attribute'].str.endswith('_RZ'), 'Attribute'].str.replace('_RZ', '')

missing_attributes = []
for idx, row in value_unknown_codes.iterrows():
    try:
        azdias.loc[azdias[row['Attribute']] == row['value'], row['Attribute']] = np.nan
    except KeyError:
        missing_attributes.append(row['Attribute']) 

Having adjusted the Attribute name in the `DIAS Attributes - Values 2017.xlsx` list of unkown value codes, now lets see how many `NaN` values are in our dataset. 

In [40]:
fig = plot_missing_values(azdias[azdias_missing_attr])
fig.show()

In [41]:
fig = plot_missing_values(azdias)
fig.show()
# py.plot(fig, filename = 'udsnd_3_missing', auto_open=False)

Next, I will move on to the remaining columns that have no matching name in the "Value Code" file and have very few `NaN` values

In [42]:
temp_missing_values = azdias[azdias_missing_attr].isna().sum()
temp_missing_values = temp_missing_values[temp_missing_values<50_000]

In [43]:
temp_missing_values

LNR                           0
CJT_KATALOGNUTZER          4854
CJT_TYP_1                  4854
CJT_TYP_2                  4854
CJT_TYP_3                  4854
CJT_TYP_4                  4854
CJT_TYP_5                  4854
CJT_TYP_6                  4854
D19_BUCH_CD                   0
D19_KONSUMTYP_MAX             0
D19_VERSI_DATUM               0
D19_VERSI_OFFLINE_DATUM       0
D19_VERSI_ONLINE_DATUM        0
KOMBIALTER                    0
RT_KEIN_ANREIZ             4854
RT_SCHNAEPPCHEN            4854
dtype: int64

**CJT_ name group**

These are values about the Customer-Journey-Typology relating to the preferred information and buying channels of consumers. Interestingly, in the `Attribute` information, we can see that this data source should be just one column with codes to represent 6 types of consumers. But in fact I can see that there are 6 columns with `_typ_x` suffix.  Perhaps these columns represent a one-hot encoded version of the existing and known column: `CJT_GESAMTTYP`? 

In [44]:
azdias[['CJT_GESAMTTYP', 'CJT_KATALOGNUTZER', 'CJT_TYP_1', 'CJT_TYP_2', 'CJT_TYP_3', 'CJT_TYP_4', 'CJT_TYP_5', 'CJT_TYP_6' ]].head(10)

,CJT_GESAMTTYP,CJT_KATALOGNUTZER,CJT_TYP_1,CJT_TYP_2,CJT_TYP_3,CJT_TYP_4,CJT_TYP_5,CJT_TYP_6
0,2.0,5.0,1.0,1.0,5.0,5.0,5.0,5.0
1,5.0,1.0,5.0,5.0,2.0,3.0,1.0,1.0
2,3.0,2.0,4.0,4.0,1.0,3.0,2.0,2.0
3,2.0,3.0,2.0,2.0,4.0,4.0,5.0,3.0
4,5.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0
5,2.0,5.0,2.0,1.0,5.0,5.0,3.0,3.0
6,5.0,3.0,5.0,5.0,1.0,3.0,1.0,2.0
7,3.0,5.0,3.0,3.0,2.0,4.0,3.0,4.0
8,3.0,4.0,4.0,4.0,1.0,3.0,4.0,5.0
9,4.0,1.0,4.0,4.0,2.0,1.0,4.0,5.0


In [45]:
# DataFrame values where the Gesamttyp values are NaN.  
azdias[['CJT_GESAMTTYP', 'CJT_KATALOGNUTZER', 'CJT_TYP_1', 'CJT_TYP_2', 'CJT_TYP_3', 'CJT_TYP_4', 'CJT_TYP_5', 'CJT_TYP_6' ]].loc[azdias['CJT_GESAMTTYP'].isna()]

,CJT_GESAMTTYP,CJT_KATALOGNUTZER,CJT_TYP_1,CJT_TYP_2,CJT_TYP_3,CJT_TYP_4,CJT_TYP_5,CJT_TYP_6
138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
890128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
890290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
890367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
890701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ok, Good learning here. I only have verification on the value code that represents `unknown` for one of the `CJT_` columns. It looks like I can see that all of the missing values from the column I **do** know are **also** missing from the remaining columns. The count of `4854` also matches two further columns we did not have value code information on but were not named `CJT_*`. 

`CJT_` columns are satisified here. 

**However**, I will note that for the next step in feature engineering the `CJT_GESAMTTYPE` (or `total_type` in english) is a _categorical_ variable and that the others seem to be a _ordinal_ value. 

In [46]:
temp_missing_values.loc[~temp_missing_values.index.str.contains('CJT_')]

LNR                           0
D19_BUCH_CD                   0
D19_KONSUMTYP_MAX             0
D19_VERSI_DATUM               0
D19_VERSI_OFFLINE_DATUM       0
D19_VERSI_ONLINE_DATUM        0
KOMBIALTER                    0
RT_KEIN_ANREIZ             4854
RT_SCHNAEPPCHEN            4854
dtype: int64

The Remaining columns that have very few `NaN` values and are not represented in our list of value codes to determine if there is a code that represents `unknown` are above.  I will deal with these columns individually: 

- `LNR`: This is something of a Unique ID that was discovered above when looking at the Pandas Profiling report. I will drop this column before makeing any predictions
- `D19_BUCH_CD` - This column was one that should have been included in the `D19_` group process above, but it has added a `_CD` suffix, which I learned from the Value Codes list that the attribute includes consumer interest  in "books and cds". I will assume the `azdias` dataset column `D19_BUCH` is equal to `D19_BUCH_CD` and use the value code associated with it to convert to `NaN` 
- `D19_KONSUMTYP_MAX` - This column nearly matchs with `D19_KONSUMTYP`, but the `_MAX` suffix could be that it has a separate meaning. I will manually match the two as I did above with `D19_BUCH_CD` as the likelihood of the same type of Value Code (`9`) is rare. The final evidence that `9` is the value of `unknown` here and should be converted to `NaN` is that the number if existing `NaN` values in `D19_KONSUMTYP` is 257113. The number of `D19_KONSUMTYP_MAX` values of `9` are 257113. Thus, I will manually add the conversion of all `D19_KONSUMTYP_MAX` values of `9` to `NaN`
- `D19_VERSI_DATUM`, `D19_VERSI_OFFLINE_DATUM`, and `D19_VERSI_ONLINE_DATUM` - These were discussed above and will be manually converted to `D19_VERSAND_*`
- `RT_KEIN_ANREIZ` and `RT_SCHNAEPPCHEN` have identical missing values to the other `CJT_` named columns and I will assume all `NaN` values are accounted for and no further conversion needs to be done. 

In [47]:
#TODO Remove LNR from list
#TODO add _CD to D19_BUCH on unknowns list
#TODO add D19_KONSUMTYP_MAX = 9 to nan list
#TODO add D19_VERSI_DATUM, D19_VERSI_OFFLINE_DATUM, and D19_VERSI_ONLINE_DATUM = 10 to the unknowns list. 

# Remove _RZ suffix from Attributes and re-run our for loop to 
# replace value codes known to represent 'missing' with np.nonlocal

value_unknown_codes.loc[value_unknown_codes['Attribute'].str.endswith('_RZ'), 'Attribute'] = value_unknown_codes.loc[value_unknown_codes['Attribute'].str.endswith('_RZ'), 'Attribute'].str.replace('_RZ', '')
value_unknown_codes.loc[value_unknown_codes['Attribute'].str.contains('BUCH'), 'Attribute'] = 'D19_BUCH_CD'
value_unknown_codes = value_unknown_codes.append(pd.DataFrame({'Attribute': ['D19_KONSUMTYP_MAX'], 'value': [9]}))
value_unknown_codes.loc[value_unknown_codes['Attribute'].str.contains('D19_VERSAND_OFFLINE_DATUM|D19_VERSAND_ONLINE_DATUM|D19_VERSAND_DATUM'), 'Attribute'] = value_unknown_codes.loc[value_unknown_codes['Attribute'].str.contains('D19_VERSAND_OFFLINE_DATUM|D19_VERSAND_ONLINE_DATUM|D19_VERSAND_DATUM'), 'Attribute'].str.replace('_VERSAND_', '_VERSI_')

missing_attributes = []
for idx, row in value_unknown_codes.iterrows():
    try:
        azdias.loc[azdias[row['Attribute']] == row['value'], row['Attribute']] = np.nan
    except KeyError:
        missing_attributes.append(row['Attribute']) 

In [48]:
fig = plot_missing_values(azdias[azdias_missing_attr])
fig.show()

## Part 0.3 Store unknown value codes list to file

I can use this list of `attributes -> Unknow Value Code` for a future cleaning process. 

In [49]:
value_unknown_codes.to_csv('value_unknown_codes.csv', index=False)

## Row based cleaning 

Next we will look for rows that are missing data.  If a single row is missing most of the data fields it won't help me make predictions.

First I will plot a histogram to see the distribution of missing values per row. 

There are 366 columns in the DataFrame. I will count the missing values in each row and look for an obivous pattern. 

As our data set was built from serveral sources. It is likely that certain individuals might be missing an entire portion of the columns. This would present a pattern.

In [50]:
len(azdias.columns)

366

In [51]:
rownulls = azdias.isnull().sum(axis=1) 
rownullsmax = rownulls.max()

In [52]:
rownulls = np.histogram(rownulls, bins=range(0, rownulls.max(), 1))

In [53]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=rownulls[1], y=rownulls[0]
    ))
fig.update_xaxes(title='Number of Missing Values')
fig.update_yaxes(title='Count of Rows')
fig.update_layout(showlegend=False, title = 'Distribution of Missing Values per Row')

In [54]:
# py.plot(fig, filename = 'udsnd_4_missing', auto_open=False)

There are three "shapes" visible in the distrobution. The first shape on the right has a steady curve. The next two shapes are skyscraper like spikes in count of individuals who are all missing that same amount of values in the row. The first curved shape to the left indicates that all of these individuals have data present in nearly 300 columns and likely some from all the disparate sources but the gradual curve of fewer and fewer missing data above 300 columns leads me to believe that the remaining data fields are truely `not applicable` such as "Age_Child_4". Fewer and fewer individuals with have data on a fourth child, becuase they have no fourth child and thus the `NaN` here is a value.

Alternatively, the large spikes both lead me to believe that individuals in these bins are missing all the data from an single data source. 

For now, I will remove rows that represent the first spike (and a little further). The second spike of missing data may present itself as a pattern in it's value in the future PCA portion. If so, I can re-adjust.

In [55]:
# A very crude way to count the number of individuals that would be removed by threshold: 
temp_len = len(azdias)
for i in [(50, "After left spike"), (215, 'Middle ground'), (278, "Before second spike"), (293, 'After second spike')]:
    print(f"{i[1]}: {temp_len - len(azdias.dropna(axis=0, how='any', thresh=i[0]))}" )

After left spike: 73517
Middle ground: 106189
Before second spike: 154831
After second spike: 320824


In [56]:
print(len(azdias.dropna(axis=0, how='any', thresh=293)))
print(len(azdias.dropna(axis=0, how='any', thresh=278)))

570397
736390


In [57]:
print(len(azdias.dropna(axis=0, how='any', thresh=293))/temp_len)
print(len(azdias.dropna(axis=0, how='any', thresh=278))/temp_len)

0.6400174591936232
0.8262709249445424


Tough call. 160,000 rows will be lost in order to be sure that each data field is truly represented. In the more strict scenario we are left with 64% of the data, but this will likely produce better results.  

**In the real world** part of this decision would be made by understanding the business case for the value of predicitons in the future. Is it more important to make a less accurate prediction with fewer data, or a more accurate prediction with fewer overall predictions possible due to missing data? 

I will start the process by including more rows and set the threshold at **278 columns**. Then, drop individual columns with a similar ratio of missing values per row. 

In [58]:
azdias.dropna(axis=0, how='any', thresh=278, inplace=True)

In [59]:
# Plot missing values now that rows have been removed.
fig = plot_missing_values(azdias)
fig.show()
# py.plot(fig, filename = 'udsnd_5_missing', auto_open=False)

Using Plotly, I can hover over data to see the value of each bar as a tooltip. Thus, I can simply look at the shape of this visual and see the drop off in rate of missing values. The first 'plateau' of values after the slow downward curve on the left are all at 22% approximately. After this 'plateau', the next level of values are at `18%`

Seeing the plateau at 22%, leads me to believe a 23% cutoff is appropriate to drop all columns with more missing values than this. 

In [60]:
temp_col_na = azdias.isna().sum()/len(azdias)
temp_col_na = temp_col_na.loc[temp_col_na > .23].index

I will use this specific list in the future cleaning function to drop these columns. I don't have enough data to train on them, therefore I will not be able to use these columns for predictions

In [61]:
# List of columns with more that 23% of values missing (after row cleaning)
temp_col_na

Index(['AGER_TYP', 'ALTER_HH', 'ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3',
       'ALTER_KIND4', 'D19_BANKEN_ANZ_12', 'D19_BANKEN_ANZ_24',
       'D19_BANKEN_DATUM', 'D19_BANKEN_DIREKT', 'D19_BANKEN_GROSS',
       'D19_BANKEN_LOKAL', 'D19_BANKEN_OFFLINE_DATUM',
       'D19_BANKEN_ONLINE_DATUM', 'D19_BANKEN_REST', 'D19_BEKLEIDUNG_GEH',
       'D19_BEKLEIDUNG_REST', 'D19_BILDUNG', 'D19_BIO_OEKO', 'D19_BUCH_CD',
       'D19_DIGIT_SERV', 'D19_DROGERIEARTIKEL', 'D19_ENERGIE', 'D19_FREIZEIT',
       'D19_GARTEN', 'D19_GESAMT_ANZ_12', 'D19_GESAMT_ANZ_24',
       'D19_GESAMT_DATUM', 'D19_GESAMT_OFFLINE_DATUM',
       'D19_GESAMT_ONLINE_DATUM', 'D19_HANDWERK', 'D19_HAUS_DEKO',
       'D19_KINDERARTIKEL', 'D19_KOSMETIK', 'D19_LEBENSMITTEL', 'D19_LOTTO',
       'D19_NAHRUNGSERGAENZUNG', 'D19_RATGEBER', 'D19_REISEN',
       'D19_SAMMELARTIKEL', 'D19_SCHUHE', 'D19_SONSTIGE', 'D19_TECHNIK',
       'D19_TELKO_ANZ_12', 'D19_TELKO_ANZ_24', 'D19_TELKO_DATUM',
       'D19_TELKO_MOBILE', 'D19_TELKO_OFFLI

In [62]:
azdias.drop(temp_col_na, axis=1, inplace=True)

## Feature Engineering

Next is a very important and challenging portion of the data cleaning needed for this project. I will need to identify columns that require transformations. Such as: 

1. Categorical columns
    - Multi-label fields
    - boolean
2. Ordinal columns
3. Date columns

Here I used the HTML file created by the Pandas Profiling tool to visually inspect columns quickly to determine which were categorical versus ordinal. This allowed me to create a file to help determine which columns should be engineered and how. 

First, I'll look at any column with an `object` datatype.


In [63]:
azdias.select_dtypes(include=['object'])

,CAMEO_DEU_2015,D19_LETZTER_KAUF_BRANCHE,EINGEFUEGT_AM,OST_WEST_KZ
1,8A,None,1992-02-10 00:00:00,W
2,4C,D19_UNBEKANNT,1992-02-12 00:00:00,W
3,2A,D19_UNBEKANNT,1997-04-21 00:00:00,W
4,6B,D19_SCHUHE,1992-02-12 00:00:00,W
5,8C,D19_ENERGIE,1992-02-12 00:00:00,W
...,...,...,...,...
891216,7A,D19_HAUS_DEKO,1992-02-10 00:00:00,W
891217,9D,D19_UNBEKANNT,1992-02-10 00:00:00,W
891218,4C,D19_BEKLEIDUNG_GEH,1992-02-10 00:00:00,W
891219,9D,D19_UNBEKANNT,1992-02-12 00:00:00,W


Looks like the `EINGEFUEGT_AM` column is a date like object. I will start with this one. As long as all values. The meaning roughly translates "inserted" and there is no information in the data attributes file about this column. I don't know what it means, but `inserted` leads me to possibly believe this is when the data was collected? It may add to the model best to separate the date into smaller chunks and test it. 

In [64]:
azdias['EINGEFUEGT_AM'].isna().sum()

0

In [65]:
# Add to cleaning function
azdias['EINGEFUEGT_AM'] = pd.to_datetime(azdias['EINGEFUEGT_AM'])
azdias['EINGEFUEGT_AM_year'] = azdias['EINGEFUEGT_AM'].dt.year
azdias['EINGEFUEGT_AM_month'] = azdias['EINGEFUEGT_AM'].dt.month
azdias.drop('EINGEFUEGT_AM', axis=1, inplace=True)

Next I will look at a categorical columns that has been combined. I noticed this one by looking through the Values .xlsx file and saw that the first value in the number pair represent one category and the second value in the number pair represents another category. For cleaning model training I will separate these values into their own columns to allow for the model to be slightly more general.

In [66]:
# Combined Category 
print(azdias['CAMEO_INTL_2015'].isna().sum())
print(azdias['CAMEO_INTL_2015'].dtype)

3250
float64


In [67]:
azdias['CAMEO_INTL_2015_0'] = azdias['CAMEO_INTL_2015']//10 
azdias['CAMEO_INTL_2015_1'] = azdias['CAMEO_INTL_2015']%10
azdias.drop('CAMEO_INTL_2015', axis=1, inplace=True)


The remaining features are categorical features which I will separate into dummie columns.

In [68]:
categorical_features = ['CAMEO_DEU_2015',
                        'CJT_GESAMTTYP',
                        'D19_KONSUMTYP',
                        'D19_LETZTER_KAUF_BRANCHE',
                        'FINANZTYP',
                        'GEBAEUDETYP',
                        'GFK_URLAUBERTYP',
                        'KBA05_MAXHERST',
                        'LP_FAMILIE_FEIN',
                        'LP_LEBENSPHASE_FEIN',
                        'LP_LEBENSPHASE_GROB',
                        'NATIONALITAET_KZ',
                        'OST_WEST_KZ',
                        'RETOURTYP_BK_S',
                        'SHOPPER_TYP',
                        'WOHNLAGE',
                        'ZABEOTYP']
                

azdias = pd.get_dummies(azdias, columns=categorical_features)

## Cleaning Function

In [69]:
def clean_AZ_dataset(dff, codes='value_unknown_codes.csv', impute=True, first_clean=False): 
    """
    Function to prepare the AZ Consumer dataset for machine learning models

    INPUT
    dff - Pandas DataFrame to be cleaned.
    codes = A file path to a predifined .csv file with list of value codes in the data that
            represent "unknown" or "missing" values. File should be formatted as such: 
            attribute, value,
            'CAMEO_DEU_2015', -1,

    RETURNS 
    LNR - Pandas series with customer numbers 
    df - Pandas DataFrame ready to be used in supervised and unsupervised models. 

    """

    print("Starting Cleaning.")
    # Read in special file to map codes to missing values.
    value_unknown_codes = pd.read_csv(codes)
    print("Loaded Missing Value codes.")

    # This is wierd, but I did this next step manually for the azdias file so that I could store it as .parquet
    # Thus, I am adding this step below for the mailout CSV's and a flag to signal that it needs to be run. 
    if first_clean == True:
        # According to data map supplied "-1" means unknown. Fill non-integer numbers with np.nan and convert to float
        dff.CAMEO_DEUG_2015 = dff.CAMEO_DEUG_2015.replace('X', np.NaN).astype(float)
        dff.CAMEO_INTL_2015 = dff.CAMEO_INTL_2015.replace('XX', np.NaN).astype(float)

    # Loop through values replaceing values with NaN.
    missing_attributes = []
    for idx, row in value_unknown_codes.iterrows():
        try:
            dff.loc[dff[row['Attribute']] == row['value'], row['Attribute']] = np.nan
        except KeyError:
            missing_attributes.append(row['Attribute']) 
    print("Finished converting missing value codes to NaN.")
    print(f"These fields were not found in the data {missing_attributes}")

    temp_len = len(dff)
    dff.dropna(axis=0, how='any', thresh=278, inplace=True)
    print(f"Dropped {temp_len-len(dff)} rows from the orginal {temp_len} rows in the dataset")

    try:
        dff.drop(['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'], axis=1, inplace=True)
    except KeyError:
        pass

    LNR = dff['LNR']

    temp_col_na = ['LNR', 'AGER_TYP', 'ALTER_HH', 'ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3',
        'ALTER_KIND4', 'D19_BANKEN_ANZ_12', 'D19_BANKEN_ANZ_24',
        'D19_BANKEN_DATUM', 'D19_BANKEN_DIREKT', 'D19_BANKEN_GROSS',
        'D19_BANKEN_LOKAL', 'D19_BANKEN_OFFLINE_DATUM',
        'D19_BANKEN_ONLINE_DATUM', 'D19_BANKEN_REST', 'D19_BEKLEIDUNG_GEH',
        'D19_BEKLEIDUNG_REST', 'D19_BILDUNG', 'D19_BIO_OEKO', 'D19_BUCH_CD',
        'D19_DIGIT_SERV', 'D19_DROGERIEARTIKEL', 'D19_ENERGIE', 'D19_FREIZEIT',
        'D19_GARTEN', 'D19_GESAMT_ANZ_12', 'D19_GESAMT_ANZ_24',
        'D19_GESAMT_DATUM', 'D19_GESAMT_OFFLINE_DATUM',
        'D19_GESAMT_ONLINE_DATUM', 'D19_HANDWERK', 'D19_HAUS_DEKO',
        'D19_KINDERARTIKEL', 'D19_KOSMETIK', 'D19_LEBENSMITTEL', 'D19_LOTTO',
        'D19_NAHRUNGSERGAENZUNG', 'D19_RATGEBER', 'D19_REISEN',
        'D19_SAMMELARTIKEL', 'D19_SCHUHE', 'D19_SONSTIGE', 'D19_TECHNIK',
        'D19_TELKO_ANZ_12', 'D19_TELKO_ANZ_24', 'D19_TELKO_DATUM',
        'D19_TELKO_MOBILE', 'D19_TELKO_OFFLINE_DATUM', 'D19_TELKO_ONLINE_DATUM',
        'D19_TELKO_REST', 'D19_TIERARTIKEL', 'D19_VERSAND_ANZ_12',
        'D19_VERSAND_ANZ_24', 'D19_VERSAND_DATUM', 'D19_VERSAND_OFFLINE_DATUM',
        'D19_VERSAND_ONLINE_DATUM', 'D19_VERSAND_REST', 'D19_VERSI_ANZ_12',
        'D19_VERSI_ANZ_24', 'D19_VERSI_DATUM', 'D19_VERSI_OFFLINE_DATUM',
        'D19_VERSI_ONLINE_DATUM', 'D19_VERSICHERUNGEN', 'D19_VOLLSORTIMENT',
        'D19_WEIN_FEINKOST', 'EXTSEL992', 'KBA05_BAUMAX', 'KK_KUNDENTYP',
        'TITEL_KZ']


    dff.drop(temp_col_na, axis=1, inplace=True)
    print("Dropped columns with missing data.")

    dff['EINGEFUEGT_AM'] = pd.to_datetime(dff['EINGEFUEGT_AM'])
    dff['EINGEFUEGT_AM_year'] = dff['EINGEFUEGT_AM'].dt.year
    dff['EINGEFUEGT_AM_month'] = dff['EINGEFUEGT_AM'].dt.month
    dff.drop('EINGEFUEGT_AM', axis=1, inplace=True)
    print('Converted EINGEFUEGT_AM to year and month')

    dff['CAMEO_INTL_2015_0'] = dff['CAMEO_INTL_2015']//10 
    dff['CAMEO_INTL_2015_1'] = dff['CAMEO_INTL_2015']%10
    dff.drop('CAMEO_INTL_2015', axis=1, inplace=True)
    print('Split CAMEO_INTL_2015 into two columns')

    categorical_features = ['CAMEO_DEU_2015',
                        'CJT_GESAMTTYP',
                        'D19_KONSUMTYP',
                        'D19_LETZTER_KAUF_BRANCHE',
                        'FINANZTYP',
                        # 'GEBAEUDETYP',
                        'GFK_URLAUBERTYP',
                        'KBA05_MAXHERST',
                        'LP_FAMILIE_FEIN',
                        'LP_LEBENSPHASE_FEIN',
                        'LP_LEBENSPHASE_GROB',
                        'NATIONALITAET_KZ',
                        'OST_WEST_KZ',
                        'RETOURTYP_BK_S',
                        'SHOPPER_TYP',
                        'WOHNLAGE',
                        'ZABEOTYP']
                

    dff = pd.get_dummies(dff, columns=categorical_features)
    print('Converted Categorical columns to dummy columns.')

    if impute == True:
        imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
        dff_imp = imp_median.fit_transform(dff)
        print("Imputed NaN values with Median")
        return LNR, pd.DataFrame(dff_imp, index=dff.index, columns=dff.columns)
    else:
        return LNR, dff

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

In [70]:
azdias_neu = pd.read_parquet('Udacity_AZDIAS_052018.parquet')
LNR, azdias_neu = clean_AZ_dataset(azdias_neu, impute=True) 

Starting Cleaning.
Loaded Missing Value codes.
Finished converting missing value codes to NaN.
These fields were not found in the data ['BIP_FLAG', 'CAMEO_DEUINTL_2015', 'D19_KK_KUNDENTYP', 'GEOSCORE_KLS7', 'HAUSHALTSSTRUKTUR', 'KBA13_CCM_1400_2500', 'SOHO_FLAG', 'WACHSTUMSGEBIET_NB', 'GEOSCORE_KLS7', 'HAUSHALTSSTRUKTUR', 'WACHSTUMSGEBIET_NB']
Dropped 154538 rows from the orginal 891221 rows in the dataset
Dropped columns with missing data.
Converted EINGEFUEGT_AM to year and month
Split CAMEO_INTL_2015 into two columns
Converted Categorical columns to dummy columns.
Imputed NaN values with Median


In [71]:
scaler = StandardScaler()

# fit transform
scaled = scaler.fit_transform(azdias_neu.astype(float))

azdias_scaled = pd.DataFrame(data=scaled, index=azdias_neu.index, columns=azdias_neu.columns)
print("original shape:   ", azdias_neu.shape)
print("scaled shape:", azdias_scaled.shape)

original shape:    (736683, 492)
scaled shape: (736683, 492)


### PCA Refresher: 

Eigenvalues - sum of squared distances from origin for the best fit line 

Singular Value of PC1 - The square root of the Eigenvalue

Eigenvector - Single unit of a PC

Loading scores - proportion of each value to get the eigenvector for the PC. 

Scree plot - Graphical representation of the percentages of variation for each Principal Component


In [72]:
pca = PCA()
pca.fit(azdias_scaled)
pca_data = pca.transform(azdias_scaled)

In [73]:
total_var = pca.explained_variance_ratio_.sum() * 100
pc_var = pca.explained_variance_ratio_

In [74]:
exp_var_cumul = np.cumsum(pc_var)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(1, len(exp_var_cumul))),
    y=exp_var_cumul,
    fill='tozeroy', # fill area between trace0 and trace1
    mode='lines', line_color='indigo'))
fig.update_layout(title = 'Cumulative Sum of Explained Variance')
fig.update_yaxes(tickformat='%', title='Explained Variance', showspikes=True)
fig.update_xaxes(title='Count of Principal Components', showspikes=True)
fig.show()
# py.plot(fig, filename='udsnd_6_pca', auto_open=False)

In [75]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=np.arange(len(pca.explained_variance_ratio_)),
        y=pca.explained_variance_ratio_
    )
)
fig.update_yaxes(tickformat=".2%", title="Percent of Explained Variance")
fig.update_xaxes(title = 'PCA Components')
fig.update_layout(title = 'Principal Component Analysis')
fig.show()
# py.plot(fig, filename='udsnd_7_pca', auto_open=False)

## Principle Component Analysis

We can see for our visuals above that like many large datasets a few components account for a large explanation of the variance. While there are a few components that are larger than the rest the largest components only account for about 25% of the total variance. 

I have seen some information about 85% being a good start, however that would be about 250 dimensions.  At 180 dimensions I get to about 75% of variance explained. I will start there. 

In [76]:
n_components = 180
pca = PCA(n_components = n_components)
pca.fit(azdias_scaled)
pca_data = pca.transform(azdias_scaled)

In [77]:
def load_score_viz(pc):
    '''
    Creates visualization of Loading scores for each feature in a principal component.

    INPUT - 
        'pc' - An integer representing the element in the list of principle components
    
    OUTPUT -
        'fig' - Plotly figure with visualization of the top 50 most impactful feature for 
                that specific component.
    '''
    loading_scores = pd.Series(pca.components_[pc], index=azdias_scaled.columns)
    sorted_loading_scores = loading_scores.abs().sort_values(ascending=False)
    fig = go.Figure(
        go.Bar(
            x = sorted_loading_scores.head(50).index,
            y= sorted_loading_scores.head(50)
        )
    )
    fig.update_yaxes(title = "Proportion of Eigenvector")
    fig.update_layout(title= f"Absolute Value of Load Score - Principle Component {pc} <br><sub>{round((pca.explained_variance_ratio_[pc] * 100), 2)}% of explained variance")
    return fig

In [78]:
fig = load_score_viz(0)
fig.show()
# py.plot(fig, filename='udsnd_8_loading', auto_open=False)

In [79]:
fig = load_score_viz(1)
fig.show()
# py.plot(fig, filename='udsnd_9_loading', auto_open=False)

In [80]:
fig = load_score_viz(2)
fig.show()
# py.plot(fig, filename='udsnd_10_loading', auto_open=False)

## Optimal Clusters

Next I will test several Kmeans centers to find a starting point of least SSE versus trainging time  

In [81]:
n_clusters = range(5,50)
scores = [MiniBatchKMeans(i, random_state=42).fit(pca_data).score(pca_data) for i in n_clusters]

In [82]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
       x=list(n_clusters),
       y=np.abs(scores),
       mode='lines+markers'
    )
)
fig.update_xaxes(title='Number of Clusters')
fig.update_yaxes(title='Sum of Squared Errors')
fig.update_layout(title='K-means Optimal Number of Cluster Centers')
fig.show()
# py.plot(fig, filename='udsnd_11_kmeans', auto_open=False)

In [83]:
# re-fith with 11 clusters
kmeans = KMeans(n_clusters=19, random_state=42)
# general population predictions
azdias_predictions = kmeans.fit_predict(pca_data)

## Read in Custome data and fit it to current clusters

Run this now on scaler, pca, and Kmeans for analysis, but later store into a single pipeline for future use.

In [84]:
cust_neu = pd.read_parquet('Udacity_CUSTOMERS_052018.parquet')
LNR, cust_neu = clean_AZ_dataset(cust_neu, impute=True)
cust_scaled = scaler.transform(cust_neu.astype(float))
cust_pca_data = pca.transform(cust_scaled)
cust_cluster_predictions = kmeans.predict(cust_pca_data)

Starting Cleaning.
Loaded Missing Value codes.
Finished converting missing value codes to NaN.
These fields were not found in the data ['BIP_FLAG', 'CAMEO_DEUINTL_2015', 'D19_KK_KUNDENTYP', 'GEOSCORE_KLS7', 'HAUSHALTSSTRUKTUR', 'KBA13_CCM_1400_2500', 'SOHO_FLAG', 'WACHSTUMSGEBIET_NB', 'GEOSCORE_KLS7', 'HAUSHALTSSTRUKTUR', 'WACHSTUMSGEBIET_NB']
Dropped 57391 rows from the orginal 191652 rows in the dataset
Dropped columns with missing data.
Converted EINGEFUEGT_AM to year and month
Split CAMEO_INTL_2015 into two columns
Converted Categorical columns to dummy columns.
Imputed NaN values with Median


In [85]:
# Cant upload Histograms to free Plotly embed site. Convert to Bar Chart
x_ax = np.unique(cust_cluster_predictions, return_counts=True)[0]
cust_clust = np.unique(cust_cluster_predictions, return_counts=True)[1]/len(cust_cluster_predictions)
az_clust = np.unique(azdias_predictions, return_counts=True)[1]/len(azdias_predictions)

In [86]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=x_ax,
        y=cust_clust,
        name='Customers'
    )
)
fig.add_trace(
    go.Bar(
        x=x_ax,
        y=az_clust,
        name='Population'
    )
)
fig.update_yaxes(tickformat='.2%', title='Frequency')
fig.update_xaxes(title='Customer Groups')
fig.update_layout(barmode='overlay', title='Customer Segmentation')
fig.update_traces(opacity=0.75)
fig.show()
# py.plot(fig, filname='udsnd_12_segments', auto_open=False)

## Understand our clusters

Now that I can see how our customer group differs from the population, I will take a look to determine some individual features that differ. 

With 366 different columns of data to start it is too exhuastive to analyse each feature. However, with PCA and our Loading Score visual, we might start with those features that had the largest proportion of the eigenvector in the components that had the greatest explained variance. 

**Cluster 18**

One cluster had more than 25% of our customer population.  Cluster 18 is a great example to start to see the features that most qualify our customers. 

In [87]:
# Add cluster column to both dataframes to filter by cluster
cust_neu['cluster'] = cust_cluster_predictions

In [88]:
def plot_feature_compare(feature, cluster):
    '''
    Builds a plot compare features of the customer group versus the population for a specific cluster.cust_neu
    
    INPUT - feature: String of column name
            cluster: integer of customer cluster to filter on
    
    OUTPUT - Plotly figure with distribution of values for each group.
    '''
    x_ax = np.unique(azdias_neu[feature], return_counts=True)[0]
    cust_mobi_reg = np.unique(cust_neu.loc[cust_neu['cluster'] == cluster, feature], return_counts=True)[1]/len(cust_neu.loc[cust_neu['cluster'] == cluster])
    az_mobi_reg = np.unique(azdias_neu[feature], return_counts=True)[1]/len(azdias_neu)
    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            x=x_ax,
            y=cust_mobi_reg,
            name='Customers'
        )
    )
    fig.add_trace(
        go.Bar(
            x=x_ax,
            y=az_mobi_reg,
            name='Population'
        )
    )
    fig.update_yaxes(tickformat='.2%', title='Frequency')
    fig.update_xaxes(title='Value')
    fig.update_layout(barmode='overlay', title=f'Customer Segmentation by Feature<br><sub><b>Cluster:</b> {cluster}  - <b>Feature:</b> {feature}')
    fig.update_traces(opacity=0.75)
    return fig

`MOBI_REGIO` - This feature had the highest loading score in the Principle component with the most explained variance. The freature describes moving patterns. Lower numbers are high mobility and high numbers are very low mobility. It is clear the Arvato Customers have lower mobility regionally.

In [89]:
fig = plot_feature_compare('MOBI_REGIO', 18)
fig.show()
# py.plot(fig, filname='udsnd_13_features', auto_open=False)

In PC2 the largest loading score was `PRAEGENDE_JUGENDJAHRE`

In [90]:
fig = plot_feature_compare('PRAEGENDE_JUGENDJAHRE', 18)
fig.show() 
# py.plot(fig, filname='udsnd_14_features', auto_open=False)

In PC3 the largest loading score was car based `KBA13_HERST_BMW_BENZ`, In the third Principle Component we can see the highest loading scores are all regarding automobiles and generally those of a luxury class. This feature is the share of BMW & Mercedes Benz within the neighborhood. A score of 5 is high.

In [91]:
fig = plot_feature_compare('KBA13_HERST_BMW_BENZ', 18)
fig.show() 
# py.plot(fig, filname='udsnd_15_features', auto_open=False)

`PLZ8_ANTG1` - Finally, the number of 1–2 family houses in the neighborhood of the individuals is lower for the customers in Cluster 18

In [92]:
fig = plot_feature_compare('PLZ8_ANTG1', 18)
fig.show() 
# py.plot(fig, filname='udsnd_16_features', auto_open=False)

## Build reusable Pipeline for Customer segmentation

I will move all the components into a single pipeline to be saved for future use on new data. 

Create a function that could be used on future customer data. 

This includes the imputed values for each column. Why? While imputed values leads to data leak in a traditional train/test data set scenario. In this case I am considering the one off scenario.  If this cluster segmentation needed to be productionized and a single user mapped to it, I could not impute values for that user. 

The transformer in this pipeline will store the median value for each feature to be able to reproduce a prediction in the case that it is used against a single row of data and that row has missing values. Of course, in our cleaning step rows with less than 70% of fields will be dropped. 

In [93]:
azdias_neu = pd.read_parquet('Udacity_AZDIAS_052018.parquet')
LNR, azdias_neu = clean_AZ_dataset(azdias_neu, impute=True) 

cust_neu = pd.read_parquet('Udacity_CUSTOMERS_052018.parquet')
LNR, cust_neu = clean_AZ_dataset(cust_neu, impute=True)

Starting Cleaning.
Loaded Missing Value codes.
Finished converting missing value codes to NaN.
These fields were not found in the data ['BIP_FLAG', 'CAMEO_DEUINTL_2015', 'D19_KK_KUNDENTYP', 'GEOSCORE_KLS7', 'HAUSHALTSSTRUKTUR', 'KBA13_CCM_1400_2500', 'SOHO_FLAG', 'WACHSTUMSGEBIET_NB', 'GEOSCORE_KLS7', 'HAUSHALTSSTRUKTUR', 'WACHSTUMSGEBIET_NB']
Dropped 154538 rows from the orginal 891221 rows in the dataset
Dropped columns with missing data.
Converted EINGEFUEGT_AM to year and month
Split CAMEO_INTL_2015 into two columns
Converted Categorical columns to dummy columns.
Imputed NaN values with Median
Starting Cleaning.
Loaded Missing Value codes.
Finished converting missing value codes to NaN.
These fields were not found in the data ['BIP_FLAG', 'CAMEO_DEUINTL_2015', 'D19_KK_KUNDENTYP', 'GEOSCORE_KLS7', 'HAUSHALTSSTRUKTUR', 'KBA13_CCM_1400_2500', 'SOHO_FLAG', 'WACHSTUMSGEBIET_NB', 'GEOSCORE_KLS7', 'HAUSHALTSSTRUKTUR', 'WACHSTUMSGEBIET_NB']
Dropped 57391 rows from the orginal 191652 rows i

In [94]:
def build_model():
    """
    Builds an Sklearn Pipeline with a Simple Imputer, Scaler, PCA, and Kmeans. 
    Input - None
    Output - Sklearn Pipeline object to be fit on Population data and reused with future data. 
    """
    transformer = FeatureUnion(
    transformer_list=[
        ('features', SimpleImputer(strategy='median')),
        ('indicators', MissingIndicator(features="all"))])
    pipeline = make_pipeline(transformer, StandardScaler(), PCA(n_components = 180), KMeans(n_clusters=19, random_state=42))
    return pipeline

pipe = build_model()
pipe.fit(azdias_neu)

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('features',
                                                 SimpleImputer(strategy='median')),
                                                ('indicators',
                                                 MissingIndicator(features='all'))])),
                ('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=180)),
                ('kmeans', KMeans(n_clusters=11, random_state=42))])

In [95]:
def save_model(model, model_filepath):
    """
    Saves model as pickle object.
    Input - model : model object
            model_filepath : filepath destination for output
    Output - None, file stored 
    """
    pkl.dump(model, open(model_filepath, 'wb'))
    pass

save_model(pipe, 'customer_segmentation/clusters.pkl')

# Part 2: Supervised Learning Model

---

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [96]:
X = pd.read_csv('Udacity_MAILOUT_052018_TRAIN.csv')
LNR, X = clean_AZ_dataset(X, first_clean=True, impute=True)
y = X['RESPONSE']
X = X.drop('RESPONSE', axis=1)

/Users/derricklewis/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning:

Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.



Starting Cleaning.
Loaded Missing Value codes.
Finished converting missing value codes to NaN.
These fields were not found in the data ['BIP_FLAG', 'CAMEO_DEUINTL_2015', 'D19_KK_KUNDENTYP', 'GEOSCORE_KLS7', 'HAUSHALTSSTRUKTUR', 'KBA13_CCM_1400_2500', 'SOHO_FLAG', 'WACHSTUMSGEBIET_NB', 'GEOSCORE_KLS7', 'HAUSHALTSSTRUKTUR', 'WACHSTUMSGEBIET_NB']
Dropped 9144 rows from the orginal 42962 rows in the dataset
Dropped columns with missing data.
Converted EINGEFUEGT_AM to year and month
Split CAMEO_INTL_2015 into two columns
Converted Categorical columns to dummy columns.
Imputed NaN values with Median


## Split

For several quick trials I will split the data  into train and test sets to verify results. 

In [97]:
X_train, X_test, y_train, y_test =  train_test_split(X, y, stratify=y)

## Evaluate Several Baseline Models

Here I will trial several types of models to determine a rough idea of feasibilty of sucess.  Below is a function to produce a ROC curve visualization for the predictions of each traine algorithm. 

In [98]:
def evaluate_model(MODEL, X_test, y_test, name):
    '''
    Function to gather basic results for printing to standard out. 
    Input - Model : trained model object
            X_test : Unseen Input features to evaluate model
            y_test : Unseen labels to evaluate model
    
    Output - Roc_auc_score for predictions
    '''
    y_pred = MODEL.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
    fig = px.area(
    x=fpr, y=tpr,
    title=f'{name}<br><sub>ROC Curve (AUC={auc(fpr, tpr):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    )
    fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
    )

    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.update_xaxes(constrain='domain')
    return fig

Some models have built in adjustments for class imbalance. I will do a quick determination of class imbalance to be used when evaluating these models. 

In [99]:
# sum(negative instances) / sum(positive instances)
class_weight = int(y.value_counts()[0]/y.value_counts()[1])
print(class_weight)

80


## Logistic Regression

In [100]:
from sklearn.linear_model import LogisticRegression
LGR = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, class_weight={0:1, 1:class_weight}, random_state=42))
LGR.fit(X_train, y_train)
fig = evaluate_model(LGR, X_test, y_test, 'Logistic Regression')
fig.show()
py.plot(fig, filname='udsnd_17_log', auto_open=False)

'https://plotly.com/~lewi0332/2147/'

## Support Vector Machines

In [101]:
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma = 'auto', class_weight={0:1, 1:class_weight}, probability=True, random_state=42))
clf.fit(X_train, y_train)
fig = evaluate_model(clf, X_test, y_test, 'Support Vector Classifier')
fig.show()
py.plot(fig, filname='udsnd_18_svm', auto_open=False)

'https://plotly.com/~lewi0332/2149/'

## ADABoost

In [102]:
model = AdaBoostClassifier(random_state=42)
model.fit(X_train, y_train)
fig = evaluate_model(model, X_test, y_test, name='AdaBoost')
fig.show()
py.plot(fig, filname='udsnd_19_ada', auto_open=False)

'https://plotly.com/~lewi0332/2151/'

## XGBoost

In [103]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='auc', scale_pos_weight = 80, random_state=42)
model.fit(X_train, y_train)
fig = evaluate_model(model, X_test, y_test, name = 'XGBoost')
fig.show()
py.plot(fig, filname='udsnd_20_xgb', auto_open=False)

'https://plotly.com/~lewi0332/2153/'

## Use XGBoost

going to use XGboost Algorithm and optimize for the best parameters. 

In [104]:
model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='auc',
    # learning_rate = 0.01,
    scale_pos_weight = 80,
    n_estimators = 20,
    max_delta_step=5
    )
param_grid = {
    'learning_rate': [.01, 0.1, 0.5, .9],
    # 'num_leaves': [2],      
    'max_depth': [2, 3, 6],
    # 'colsample_bytree':[0.5, 1.0],
    # "min_data_in_leaf":[20, 100],
    # 'min_child_samples': [0, 50],
    # 'max_bin': [100, 1000],
    # 'reg_lambda': [1e-9, 1.0],
    # 'reg_alpha': [1e-9, 1.0],
    # 'scale_pos_weight': [80],
    'n_estimators': [20,40,60]
}
grid = GridSearchCV(model, param_grid, cv=4, scoring='roc_auc', verbose=3)
grid.fit(X, y)
# evaluate_model(grid, X_test, y_test)

Fitting 4 folds for each of 36 candidates, totalling 144 fits
[CV 1/4] END learning_rate=0.01, max_depth=2, n_estimators=20;, score=0.775 total time=   1.8s
[CV 2/4] END learning_rate=0.01, max_depth=2, n_estimators=20;, score=0.739 total time=   1.7s
[CV 3/4] END learning_rate=0.01, max_depth=2, n_estimators=20;, score=0.778 total time=   1.7s
[CV 4/4] END learning_rate=0.01, max_depth=2, n_estimators=20;, score=0.736 total time=   1.7s
[CV 1/4] END learning_rate=0.01, max_depth=2, n_estimators=40;, score=0.759 total time=   2.6s
[CV 2/4] END learning_rate=0.01, max_depth=2, n_estimators=40;, score=0.736 total time=   2.6s
[CV 3/4] END learning_rate=0.01, max_depth=2, n_estimators=40;, score=0.778 total time=   2.5s
[CV 4/4] END learning_rate=0.01, max_depth=2, n_estimators=40;, score=0.737 total time=   2.5s
[CV 1/4] END learning_rate=0.01, max_depth=2, n_estimators=60;, score=0.769 total time=   3.6s
[CV 2/4] END learning_rate=0.01, max_depth=2, n_estimators=60;, score=0.735 total t

GridSearchCV(cv=4,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, eval_metric='auc',
                                     gamma=None, gpu_id=None,
                                     importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=5,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=20, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=80, subsample=None,
    

In [105]:
grid.best_score_

0.7743110842577072

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter.

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [108]:
mailout_test = pd.read_csv('Udacity_MAILOUT_052018_TEST.csv', sep=';')
# LNR, X = clean_AZ_dataset(mailout_test, first_clean=False, impute=True)

In [ ]:
y_pred = grid.predict_proba(X)[:, 1]

In [ ]:
kaggle_sub = pd.DataFrame(index=mailout_test['LNR'].astype('int32'), data=preds)
kaggle_sub.rename(columns={0: "RESPONSE"}, inplace=True)
kaggle_sub.to_csv('kaggle_sub.csv')